# Sample based on Qiskit First Machine Learning Programming Experiment
https://github.com/Qiskit/qiskit-machine-learning

In [ ]:
# Restart the kernel after running that cell
!pip install qiskit-machine-learning qiskit-terra==0.20.1

In [10]:
from qiskit import *
from qiskit.utils import QuantumInstance

# Choose between various Azure Quantum backends available
See how to add Azure Quantum Providers at https://learn.microsoft.com/azure/quantum/how-to-add-a-provider

In [4]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = "",
    location = ""
)

# Create IonQ simulator and QPU backends
ionq_simulator_backend = provider.get_backend("ionq.simulator")
ionq_qpu_backend = provider.get_backend("ionq.qpu")

# Create Rigetti simulator and QPU backends
rigetti_simulator_backend = provider.get_backend("rigetti.sim.qvm")
rigetti_qpu_backend = provider.get_backend("rigetti.qpu.aspen-m-3")

# Create Quantinuum simulator and QPU backends
quantinuum_simulator_backend = provider.get_backend("quantinuum.sim.h1-2e")
quantinuum_qpu_backend = provider.get_backend("quantinuum.qpu.h1-2")

backend = ionq_simulator_backend


# Straight from the Qiskit Machine Learning sample
Only addition is the QuantumInstance when creating the VQC object: quantum_instance=QuantumInstance(backend)

In [ ]:
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.datasets import ad_hoc_data

seed = 1376
algorithm_globals.random_seed = seed

# Use ad hoc data set for training and test data
feature_dim = 2  # dimension of each data point
training_size = 20
test_size = 10

# training features, training labels, test features, test labels as np.ndarray,
# one hot encoding for labels
training_features, training_labels, test_features, test_labels = ad_hoc_data(
    training_size=training_size, test_size=test_size, n=feature_dim, gap=0.3
)

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement="linear")
ansatz = TwoLocal(feature_map.num_qubits, ["ry", "rz"], "cz", reps=3)
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=COBYLA(maxiter=100),
    quantum_instance=QuantumInstance(backend),
)
vqc.fit(training_features, training_labels)

score = vqc.score(test_features, test_labels)
print(f"Testing accuracy: {score:0.2f}")